In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [1]:
import numpy as np
import torch as th
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

In [2]:
root = './data'
download = False # download MNIST dataset or not
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = dset.MNIST(root=root, train=True, transform=trans, download=download)
test_set = dset.MNIST(root=root, train=False, transform=trans)

batch_size = 100

train_loader = th.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = th.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
shuffle=False)

In [25]:
class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def classify(self, x):
        x = self.forward(x)
        x = x.data.numpy()
        return np.argmax(x, axis=1)
    
    def name(self):
        return "MLP"

In [71]:
model = MLPNet()

In [72]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()
epochs = 10

In [73]:
for it in range(epochs):
    for batch_idx, (x, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            x = Variable(x)
            labels = Variable(labels)
            out = model(x)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
            if (batch_idx + 1) % 100 == 0:
                print(f'Loss at {batch_idx + 1} was {loss.data[0]}')
            
    

Loss at 100 was 0.8169337511062622
Loss at 200 was 0.40917372703552246
Loss at 300 was 0.3781321048736572
Loss at 400 was 0.30947887897491455
Loss at 500 was 0.38027310371398926
Loss at 600 was 0.3209952414035797
Loss at 100 was 0.18924345076084137
Loss at 200 was 0.14913547039031982
Loss at 300 was 0.17049507796764374
Loss at 400 was 0.20979554951190948
Loss at 500 was 0.1809203326702118
Loss at 600 was 0.28378087282180786
Loss at 100 was 0.159821555018425
Loss at 200 was 0.24384194612503052
Loss at 300 was 0.26681187748908997
Loss at 400 was 0.14975062012672424
Loss at 500 was 0.1507555991411209
Loss at 600 was 0.15071320533752441
Loss at 100 was 0.2385449856519699
Loss at 200 was 0.1758805215358734
Loss at 300 was 0.15935739874839783
Loss at 400 was 0.21230049431324005
Loss at 500 was 0.05583734065294266
Loss at 600 was 0.09511061757802963
Loss at 100 was 0.17783765494823456
Loss at 200 was 0.10900569707155228
Loss at 300 was 0.05071485415101051
Loss at 400 was 0.15149395167827606
L

In [74]:
evaluate_x = Variable(test_loader.dataset.test_data.type_as(th.FloatTensor()))

In [75]:
evaluate_y = Variable(test_loader.dataset.test_labels.type_as(th.LongTensor()))

In [76]:
output = model.classify(evaluate_x)

In [77]:
metrics.accuracy_score(output, evaluate_y.data.numpy())

0.91810000000000003